In [ ]:
from google.colab import drive
drive.mount('/gdrive')

In [ ]:
!pip install datasets

In [ ]:
from datasets import load_dataset

dataset = load_dataset("jnlpba")

In [ ]:
task = "ner" 
model_checkpoint = "distilbert-base-uncased"
batch_size = 8

In [ ]:
dataset

In [ ]:
dataset["validation"][0]

In [ ]:
dataset["train"][0]

In [ ]:
label_list = dataset["train"].features[f"ner_tags"].feature.names
print(label_list)

In [ ]:
label_list2 = dataset["validation"].features[f"ner_tags"].feature.names
print(label_list2)

In [ ]:
from datasets import load_dataset
train = load_dataset("jnlpba", split="train")

In [ ]:

from datasets import load_dataset
validation = load_dataset("jnlpba", split="validation")

In [ ]:
count =0
count_2 =0
count_3 = 0
count_4 = 0
count_5 = 0
count_6 = 0
count_7 = 0
count_8 = 0
count_9 = 0
count_10 = 0
for rows in validation['ner_tags']:
  for num in rows:
    if num == 1:
      count +=1
    elif num == 2:
      count_2 +=1
    elif num == 3:
      count_3 +=1
    elif num == 4:
      count_4 +=1
    elif num == 5:
      count_5 +=1
    elif num == 6:
      count_6 +=1
    elif num == 7:
      count_7 +=1
    elif num == 8:
      count_8 +=1
    elif num == 9:
      count_9 +=1
    elif num == 10:
      count_10 +=1
    
print("Number of 1 tags in dataset: ",count)
print("Number of 2 tags in dataset:",count_2)
print("Number of 3 tags in dataset:",count_3)
print("Number of 4 tags in dataset:",count_4)
print("Number of 5 tags in dataset:",count_5)
print("Number of 6 tags in dataset:",count_6)
print("Number of 7 tags in dataset:",count_7)
print("Number of 8 tags in dataset:",count_8)
print("Number of 9 tags in dataset:",count_9)
print("Number of 10 tags in dataset:",count_10)

In [ ]:
import pandas
train.set_format("pandas")
train[0]


In [ ]:
train

In [ ]:
train.__getitem__(0)

train.set_format("pandas")

train.__getitem__(0)

NameError: ignored

In [ ]:
df = train.to_pandas()
df.head()

In [ ]:
df1 = validation.to_pandas()
df1.head(1)

In [ ]:
train['tokens']

In [ ]:
train.shape

In [ ]:

df.head()

In [ ]:



import string

In [ ]:
train

In [ ]:
text_file = train['tokens']
flat_list=[]
for sublist in text_file:
    for item in sublist:
        flat_list.append(item)


textfile = open("test_tokens.txt", "w")
for element in flat_list:
    textfile.write(element + "\n")
textfile.close()


!wordcloud_cli --text ./test_tokens.txt --imagefile test_wordcloud.png

In [ ]:
!pip install transformers

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased') 

In [ ]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

In [ ]:
example = dataset['validation'][0]
print(example["tokens"])

In [ ]:
train[0]

In [ ]:
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
tokens

In [ ]:
print(tokenized_input.word_ids())

In [ ]:
word_ids = tokenized_input.word_ids()
aligned_labels = [-100 if i is None else example[f"{task}_tags"][i] for i in word_ids]
print(len(aligned_labels), len(tokenized_input["input_ids"]))

In [ ]:
label_all_tokens = True

In [ ]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"{task}_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            
            if word_idx is None:
                label_ids.append(-100)
            
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
           
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
tokenize_and_align_labels(dataset['train'][:3])

In [ ]:
tokenized_datasets = dataset.map(tokenize_and_align_labels, batched=True)

In [ ]:
tokenized_datasets

In [ ]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))

In [ ]:

# model_name = model_checkpoint.split("/")[-1]
# args = TrainingArguments(
#     f"{model_name}-finetuned-{task}",
#     evaluation_strategy = "epoch",
#     learning_rate=2e-5,
#     per_device_train_batch_size=batch_size,
#     per_device_eval_batch_size=batch_size,
#     num_train_epochs=50,
#     weight_decay=0.01,
#     push_to_hub=True,
# )

In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [ ]:
!pip install seqeval

In [ ]:
from datasets import load_dataset, load_metric
metric = load_metric("seqeval")

In [ ]:
labels = [label_list[i] for i in example[f"{task}_tags"]]
metric.compute(predictions=[labels], references=[labels])

In [ ]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

training api

In [ ]:
# import os
# os.listdir("/gdrive/MyDrive/jnlpba")

In [ ]:
training_args = TrainingArguments(
    output_dir="/gdrive/MyDrive/jnlpba/",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=20,
    weight_decay=0.01,
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `DistilBertForTokenClassification.forward` and have been ignored: tokens, ner_tags, id. If tokens, ner_tags, id are not expected by `DistilBertForTokenClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 37094
  Num Epochs = 20
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 92740


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.197100,0.308482,0.703487,0.806244,0.751369,0.904193
2,0.139400,0.353866,0.718781,0.808831,0.761152,0.908730


Saving model checkpoint to /gdrive/MyDrive/jnlpba/checkpoint-500
Configuration saved in /gdrive/MyDrive/jnlpba/checkpoint-500/config.json
Model weights saved in /gdrive/MyDrive/jnlpba/checkpoint-500/pytorch_model.bin
tokenizer config file saved in /gdrive/MyDrive/jnlpba/checkpoint-500/tokenizer_config.json
Special tokens file saved in /gdrive/MyDrive/jnlpba/checkpoint-500/special_tokens_map.json
Saving model checkpoint to /gdrive/MyDrive/jnlpba/checkpoint-1000
Configuration saved in /gdrive/MyDrive/jnlpba/checkpoint-1000/config.json
Model weights saved in /gdrive/MyDrive/jnlpba/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in /gdrive/MyDrive/jnlpba/checkpoint-1000/tokenizer_config.json
Special tokens file saved in /gdrive/MyDrive/jnlpba/checkpoint-1000/special_tokens_map.json
Saving model checkpoint to /gdrive/MyDrive/jnlpba/checkpoint-1500
Configuration saved in /gdrive/MyDrive/jnlpba/checkpoint-1500/config.json
Model weights saved in /gdrive/MyDrive/jnlpba/checkpoint-

In [ ]:
trainer.save_model()

In [ ]:
trainer.evaluate()

In [ ]:
dataset['validation']

In [ ]:
pred = trainer.predict(dataset['validation'])
print(pred)

In [ ]:
def get_spans(i, tokenized_datasets, id2ner, subset='validation'):
  tokens = tokenized_datasets['validation'][i]['tokens']
  label1 = tokenized_datasets['validation'][i]['ner_tags']
  text = "".join(tokens)
  
  lis =list(map(len, tokens))
  # lis1 = reduce(lambda x, y: x + y, lis)
  # # print(lis1)
  arr = np.array(lis)
  csum = np.cumsum(arr)
  # print(csum)
  # print(csum)
  arr1 = np.hstack((csum[1:-1], np.sum(arr) ))
  # arr1 = np.hstack((np.array([0]), csum[1:-1] ))
  # print(arr1)
  entities = [id2ner[x] if x != -100 else "STR" for x in label1 ]
  start = list(zip(csum.tolist(), arr1.tolist(), entities))
  # print(text)
  # print(len(start))
  # print(start)

  
  return start, text
  

In [ ]:
def get_names(tokens_names, tokenized_datasets, id2ner, subset='test'):
  # span[2]
  full_name_list = []
  for i in range(len(tokenized_datasets[subset])):
    # print('1')
    span_i, text_i = get_spans(i, tokenized_datasets, id2ner, subset)
    name_list =[]
    for s in span_i:
      # print(s)
      # print('2')
      if s[2].endswith(tokens_names):
        name_i = text_i[s[0]:s[1]]
        print(name_i)
        # name_list = name_list.append(name_i)
        # "".join(name_list).strip()
  #   full_name_list = full_name_list.append(name_list)
  # print(full_name_list)

In [ ]:
star , text1 = get_spans(1, tokenized_datasets, label_list, subset='validation')
print(star)

In [ ]:
def func(span):
  out_span = []

  for i in range(len(span)-1):
    out_span.append((span[i][0], span[i][1], span[i+1][2]))
  return out_span


In [ ]:
!pip install pip install ipymarkup


In [ ]:
from ipymarkup import show_span_box_markup, show_dep_ascii_markup

span, text = get_spans(1, tokenized_datasets, label_list, subset='validation')
o_span  = func(span)
# print(span)
# print(text)
show_span_box_markup(text, o_span)

In [ ]:
tokenized_datasets['validation']['tokens'][1]